In [1]:
%%capture
!pip install rdkit

In [2]:
#bunch of cool stuff
import pandas as pd
import numpy as np
import os
import rdkit
import io
from PIL import Image
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem import AllChem

In [3]:
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 800,800

In [4]:
!git clone https://github.com/AstraZeneca/peptide-tools.git # get the peptide-tools repository to use the SMARTS .dat file
substructures_smarts_file = '/content/peptide-tools/pIChemiSt/pichemist/data/smarts/legacy/smarts_pKaMatcher.dat' # set the path to the SMARTS file

Cloning into 'peptide-tools'...
remote: Enumerating objects: 1392, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1392 (delta 44), reused 53 (delta 42), pack-reused 1324 (from 1)
Receiving objects: 100% (1392/1392), 588.36 KiB | 8.65 MiB/s, done.
Resolving deltas: 100% (817/817), done.


In [7]:
def ionizable_groups(SMILES):
  matches = {} # where the ion centers will be stored. I tried to save the indices to highlight them but some of the positions defined in the pKaMatcher file are wrong. To-do!
  compound_mol = Chem.MolFromSmiles(SMILES)
  global ion_centers
  ion_centers = []

  with open(substructures_smarts_file, 'r') as f:
    for line in f.readlines(): # read the file line by line
      ln = line.strip().split()
      if not ln or line.startswith('#'): continue # skip empty lines and comments

      substr_mol = Chem.MolFromSmarts(ln[1]) # get the substructure in the current line as Mol
      substr_matches = compound_mol.GetSubstructMatches(substr_mol) # tuple of matches with the inputed molecule

      if len(substr_matches) != 0: # if the matches tuple has at least one item
        print(ln[1])
        for substr in range(len(substr_matches)): # for every matched substructure
          print(f"Ciclo {substr}")
          print(substr_matches[substr])
          compound_center = substr_matches[substr][int(ln[2])-1] # get the atom index for the ionization center
          print(int(ln[2]))
          if compound_center in ion_centers: # if the center has already been added, skip it
            continue
          else: # if it hasn't been added
            ion_centers.append(compound_center) # add it for subsequent checks
            matches[compound_center] = ln[5] # add the center and its type to the matches dictionary
            if len(ln) > 6: # this checks if the substructure has more than one ionization center (e.g. catechol-like compounds)
              compound_center = substr_matches[substr][int(ln[6])-1] # get the atom index for the second ionization center
              ion_centers.append(compound_center) # add it for subsequent checks
              matches[compound_center] = ln[9] # add the center and its type to the matches dictionary

  total_acid = 0
  total_base = 0
  for key, value in matches.items():
    if value == 'acid':
      total_acid += 1
    elif value == 'base':
      total_base += 1


  if total_acid + total_base > 0:
    ions = "Ionizable"
  else:
    ions = "Non-ionizable"

  return ions

In [9]:
#test = 'CC(C)CC(C(=O)NC(CC(=O)O)C(=O)NC(CC(C)C)C(=O)O)NC(=O)C(C(C)C)NC(=O)C(CCCCN)NC(=O)C(CC(=O)N)NC(=O)C(CCSC)NC(=O)C(CC(=O)O)NC(=O)C(CC1=CN=CN1)N ' # this is an example to try different SMILES
test = 'CN1CCC23C4C1CC5=C2C(=C(C=C5)O)OC3C(C=C4)O '
ionizable_groups(test)

[OH1][c]1[c](-[O])[c][c][c][c]1
Ciclo 0
(14, 11, 10, 15, 9, 8, 13, 12)
1
[OH1:1][c:2]1[c:3][c:4][c:5][c:6][c:7]1
Ciclo 0
(14, 11, 10, 9, 8, 13, 12)
1
[NX3+0;H0;!$(NC=[!#6]);!$(NC#[!#6]):1][#6:2]
Ciclo 0
(1, 0)
1
Ciclo 1
(1, 2)
1
Ciclo 2
(1, 6)
1


'Ionizable'

In [ ]:
lista = ('a')
len(lista)
for i in range(len(lista)):
  print(i)

0


In [11]:
df = pd.read_csv('NAPRORE_v4.2.csv')

df['IONIZABLE'] = df['SMILES'].apply(lambda x: ionizable_groups(x))

Streaming output truncated to the last 5000 lines.
(64, 63, 61, 59, 58, 57, 65)
1
[OH1][c]1[c](-[O])[c][c][c][c]1
Ciclo 0
(22, 21, 2, 1, 3, 4, 23, 20)
1
[OH1:1][c:2]1[c:3][c:4][c:5][c:6][c:7]1
Ciclo 0
(22, 21, 20, 23, 4, 3, 2)
1
[OH1][c]1[c](-[O])[c][c][c][c]1
Ciclo 0
(13, 12, 2, 1, 3, 4, 10, 11)
1
[OH1:1][c:2]1[c:3][c:4][c:5][c:6][c:7]1
Ciclo 0
(13, 12, 11, 10, 4, 3, 2)
1
[CX3](=O)[OX2H1]
Ciclo 0
(7, 8, 9)
2
[C,c,N,n:1](=[O,S:2])-[SX2H1,OX2H1:3]
Ciclo 0
(7, 8, 9)
2
[OH1][c]1[c](-[O])[c][c][c][c]1
Ciclo 0
(22, 21, 23, 24, 25, 18, 19, 20)
1
Ciclo 1
(37, 36, 2, 1, 3, 4, 34, 35)
1
[OH1:1][c:2]1[c:3][c:4][c:5][c:6][c:7]1
Ciclo 0
(22, 21, 20, 19, 18, 25, 23)
1
Ciclo 1
(24, 23, 21, 20, 19, 18, 25)
1
Ciclo 2
(37, 36, 35, 34, 4, 3, 2)
1
[CX3](=O)[OX2H1]
Ciclo 0
(26, 27, 28)
2
[C,c,N,n:1](=[O,S:2])-[SX2H1,OX2H1:3]
Ciclo 0
(26, 27, 28)
2
[OH1][c]1[c](-[O])[c][c][c][c]1
Ciclo 0
(8, 7, 9, 10, 11, 4, 5, 6)
1
[OH1:1][c:2]1[c:3][c:4][c:5][c:6][c:7]1
Ciclo 0
(8, 7, 6, 5, 4, 11, 9)
1
Ciclo 1
(10, 9, 7,

In [12]:
df

,Unnamed: 0.1,Unnamed: 0,NAPRORE_ID,SMILES,NAME,SOURCE_SPECIES,REFERENCE,NPC_PATHWAY_1,NPC_PATHWAY_2,NPC_PATHWAY_3,...,MOL_WT,HBD_COUNT,HBA_COUNT,ROTABLE_BONDS,LOGP,TPSA,AROMATIC_RING_COUNT,ACID_GROUP_COUNT,BASE_GROUP_COUNT,IONIZABLE
0,0,0,1,OC(C(C(C(COC(c(c(-c(c(O)c1O)c2cc1O)c1O)cc(O)c1...,['Gemin D'],['Phyllanthus acuminatus'],['https://doi.org/10.3390/molecules26216493'],Shikimates and Phenylpropanoids,NaN,NaN,...,634.455,11,18,5,-0.45390,318.50,3,9,0,Ionizable
1,1,1,2,O=C(OC1OC(CO)C2OC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(...,['Phyllanemblinin B'],['Phyllanthus acuminatus'],['https://doi.org/10.3390/molecules26216493'],Shikimates and Phenylpropanoids,NaN,NaN,...,634.455,11,18,3,-0.29650,310.66,3,9,0,Ionizable
2,2,2,3,O=C(OC1OC2COC(=O)c3cc(O)c(O)c(O)c3-c3c(cc(O)c(...,['Corilagin'],['Phyllanthus acuminatus'],['https://doi.org/10.3390/molecules26216493'],Shikimates and Phenylpropanoids,NaN,NaN,...,634.455,11,18,2,-0.29650,310.66,3,9,0,Ionizable
3,3,3,4,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2c1c(...,['Prodelphinidin B dimer'],['Phyllanthus acuminatus'],['https://doi.org/10.3390/molecules26216493'],Shikimates and Phenylpropanoids,NaN,NaN,...,610.524,12,14,3,2.40620,261.22,4,11,0,Ionizable
4,4,4,5,Oc1cc(O)c2c(c1)OC(c1cc(O)c(O)c(O)c1)C(O)C2,['(epi)galocatequina'],['Phyllanthus acuminatus'],['https://doi.org/10.3390/molecules26216493'],Shikimates and Phenylpropanoids,NaN,NaN,...,306.270,6,7,1,1.25170,130.61,2,5,0,Ionizable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
928,928,928,929,CC1=C2CCC3(C)OC3C2C(C)(C)CCC1,['himachalene epoxide'],['Manekia naranjoana'],['https://doi.org/10.1177/1934578X1100600135'],Terpenoids,NaN,NaN,...,220.356,0,1,0,4.08050,12.53,0,0,0,Non-ionizable
929,929,929,930,C=C1CCC[C@]2(C)Cc3occ(C)c3C[C@@H]12,['atractylone'],['Manekia naranjoana'],['https://doi.org/10.1177/1934578X1100600135'],Terpenoids,NaN,NaN,...,216.324,0,1,0,4.04922,13.14,1,0,0,Non-ionizable
930,930,930,931,C/C(=C/CC[C@]1(C)C2C[C@@H]3[C@H](C2)C31C)CO,['(Z)-α-santalol'],['Manekia naranjoana'],['https://doi.org/10.1177/1934578X1100600135'],Terpenoids,NaN,NaN,...,220.356,1,1,4,3.38740,20.23,0,0,0,Non-ionizable
931,931,931,932,C/C1=C\CC/C(C=O)=C\[C@@H]2[C@H](CC1)C2(C)C,['isobicyclogermacrenal'],['Manekia naranjoana'],['https://doi.org/10.1177/1934578X1100600135'],Terpenoids,NaN,NaN,...,218.340,0,1,1,3.90420,17.07,0,0,0,Non-ionizable


In [ ]:
df.to_csv('NAPRORE_v4.2.csv')